In [ ]:
# Basic inventory and user data
inventory = {}
users = {"admin": "admin123"}  # username: password
sales = []
def authenticate(username, password):
    return users.get(username) == password

def add_product(pid, name, quantity, price):
    if pid in inventory:
        return "Product ID already exists."
    if not (quantity.isdigit() and price.replace('.', '', 1).isdigit()):
        return "Invalid quantity or price."
    inventory[pid] = {"name": name, "quantity": int(quantity), "price": float(price)}
    return "Product added."

def edit_product(pid, name, quantity, price):
    if pid not in inventory:
        return "Product not found."
    inventory[pid] = {"name": name, "quantity": int(quantity), "price": float(price)}
    return "Product updated."

def delete_product(pid):
    if pid in inventory:
        del inventory[pid]
        return "Product deleted."
    return "Product not found."

def view_inventory():
    return inventory

def low_stock(threshold=5):
    return {pid: info for pid, info in inventory.items() if info["quantity"] <= threshold}

def record_sale(pid, qty):
    if pid not in inventory or inventory[pid]["quantity"] < qty:
        return "Not enough stock."
    inventory[pid]["quantity"] -= qty
    sales.append({"pid": pid, "qty": qty, "total": qty * inventory[pid]["price"]})
    return "Sale recorded."

def sales_summary():
    total_items = sum(sale["qty"] for sale in sales)
    total_earned = sum(sale["total"] for sale in sales)
    return f"Items Sold: {total_items}, Total Earned: ${total_earned:.2f}"
import tkinter as tk
from tkinter import messagebox

class InventoryApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Inventory Management")
        self.logged_in = False
        self.create_login_screen()

    def create_login_screen(self):
        self.clear_window()
        tk.Label(self.root, text="Login", font=("Arial", 16)).pack()
        self.username_entry = tk.Entry(self.root)
        self.username_entry.pack()
        self.password_entry = tk.Entry(self.root, show="*")
        self.password_entry.pack()
        tk.Button(self.root, text="Login", command=self.login).pack()

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        if authenticate(username, password):
            self.logged_in = True
            self.create_main_screen()
        else:
            messagebox.showerror("Error", "Invalid credentials")

    def create_main_screen(self):
        self.clear_window()
        tk.Label(self.root, text="Inventory System", font=("Arial", 16)).pack()
        buttons = [
            ("Add Product", self.add_product_screen),
            ("Edit Product", self.edit_product_screen),
            ("Delete Product", self.delete_product_screen),
            ("View Inventory", self.show_inventory),
            ("Low Stock Alert", self.show_low_stock),
            ("Record Sale", self.record_sale_screen),
            ("Sales Summary", self.show_sales_summary),
        ]
        for text, command in buttons:
            tk.Button(self.root, text=text, command=command).pack(fill='x')

    def add_product_screen(self):
        self.input_screen("Add Product", add_product)

    def edit_product_screen(self):
        self.input_screen("Edit Product", edit_product)

    def delete_product_screen(self):
        self.simple_input_screen("Delete Product", delete_product, "Product ID")

    def record_sale_screen(self):
        self.simple_input_screen("Record Sale", lambda pid_qty: record_sale(pid_qty[0], int(pid_qty[1])), "Product ID", "Quantity")

    def show_inventory(self):
        self.show_text_screen("Inventory", view_inventory())

    def show_low_stock(self):
        self.show_text_screen("Low Stock", low_stock())

    def show_sales_summary(self):
        self.show_text_screen("Sales Summary", sales_summary())

    def input_screen(self, title, func):
        self.clear_window()
        tk.Label(self.root, text=title, font=("Arial", 14)).pack()
        pid = tk.Entry(self.root)
        name = tk.Entry(self.root)
        qty = tk.Entry(self.root)
        price = tk.Entry(self.root)
        for e, lbl in zip([pid, name, qty, price], ["ID", "Name", "Qty", "Price"]):
            tk.Label(self.root, text=lbl).pack()
            e.pack()
        def on_submit():
            msg = func(pid.get(), name.get(), qty.get(), price.get())
            messagebox.showinfo("Result", msg)
        tk.Button(self.root, text="Submit", command=on_submit).pack()
        tk.Button(self.root, text="Back", command=self.create_main_screen).pack()

    def simple_input_screen(self, title, func, *fields):
        self.clear_window()
        tk.Label(self.root, text=title, font=("Arial", 14)).pack()
        entries = []
        for f in fields:
            tk.Label(self.root, text=f).pack()
            e = tk.Entry(self.root)
            e.pack()
            entries.append(e)
        def on_submit():
            values = [e.get() for e in entries]
            msg = func(values if len(values) > 1 else values[0])
            messagebox.showinfo("Result", msg)
        tk.Button(self.root, text="Submit", command=on_submit).pack()
        tk.Button(self.root, text="Back", command=self.create_main_screen).pack()

    def show_text_screen(self, title, data):
        self.clear_window()
        tk.Label(self.root, text=title, font=("Arial", 14)).pack()
        text = tk.Text(self.root, height=20, width=50)
        text.pack()
        text.insert(tk.END, str(data))
        tk.Button(self.root, text="Back", command=self.create_main_screen).pack()

    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()
# Run the app in Jupyter
root = tk.Tk()
app = InventoryApp(root)
root.mainloop()
